# Setup

### SKEMA Text Reading Setup

The SKEMA Text Reading pipeline is currently running on a server that is not yet world-accessible. This is a reminder to for the host to set up port-forwarding using the following command (this will only work for those who already have ssh-tunneling set up; contact `clayton@lum.ai` or `enoriega@arizona.edu` for instructions):

`ssh -L 9000:localhost:9000 leviathan`

### SKEMA FUNMAN Setup

- Install graphviz, pkg-config, z3
  - See README.md in funman: https://github.com/ml4ai/funman
- Clone and install the 
  - `git clone https://github.com/ml4ai/Model2SMTLib.git`
  - `cd Model2SMTLib`
  - `pip install -e .`
- Clone and install the `funman` module
  - `git clone https://github.com/ml4ai/funman.git`
  - `cd funman`
  - `pip install -e .`

# Package Load

In [ ]:
import ipywidgets as widgets
import shutil
from ipywidgets import interact
from ipywidgets import interactive
import json
import gpt3 as gpt
from openai import OpenAIError
from IPython import display
from base64 import b64decode
from pprint import PrettyPrinter
import urllib.parse
import gromet as gromet
import requests
from datetime import datetime
import importlib
import matplotlib.pyplot as plt
import plotly.express as px
plt.style.use("fivethirtyeight")
import pandas as pd
import sys
import uuid
import os
import requests

# SKEMA TR pipeline
from typing import Any, List, Mapping, Optional
from pandas import DataFrame

# SKEMA FUNMAN
from funman.funman import Funman
from funman.scenario import ParameterSynthesisScenario
from funman.model import Parameter
from funman.scenario import ParameterSynthesisScenarioResult
from funman.parameter_space import ParameterSpace
from funman.search_utils import SearchConfig
#%matplotlib inline

# Demo Setup Parameters

In [ ]:
XDD_GET_URL = "https://xdddev.chtc.io/askem/object/"
XDD_CREATE_URL = "https://xdddev.chtc.io/askem/create"
XDD_ARTICLES_URL = "https://xdd.wisc.edu/api/"
MIT_XDD_KEY = "81622ba9-b82d-4128-8eb3-bec123d03979"

ppr=PrettyPrinter()


#MODEL_BASE = "src.model"
DYNAMIC_MODEL_DIR = "dynmodels"
if not os.path.exists(DYNAMIC_MODEL_DIR):
    os.mkdir(DYNAMIC_MODEL_DIR)
DYNAMIC_MODEL_BASE = DYNAMIC_MODEL_DIR.replace("/", ".")

#originalModelModuleName = MODEL_BASE + "." + "CHIME_SIR_default_model"
#originalModelFileName = MODEL_BASE.replace(".", "/") + "/" + "CHIME_SIR_default_model" + ".py"


# Demo Functions

### Model Load, Mutate, Visualize, Store

In [ ]:
def storeModelSourceAndLoad(modelSrcTxt, askemMetadata):
    newModelFilename = str(uuid.uuid4())
    newModelFQFilename = DYNAMIC_MODEL_DIR + "/" + newModelFilename + ".py"
    newModelModuleName = DYNAMIC_MODEL_BASE + "." + newModelFilename

    with open(newModelFQFilename, "w") as fout:
        for line in modelSrcTxt:
            fout.write(line)
        fout.close()
    
    print("NEW MODULE NAME", newModelModuleName)
    newModule = importlib.import_module(newModelModuleName)
    newModule.__ASKEMMETADATA__ = askemMetadata
    return newModule


In [ ]:
#
# This code will create a clone of the original model (called originalModelModuleName).
# It will mutate the new model s.t. if "constantLineNo" is >=0, it will replace oldConstant with newConstant
# Then it will return a newly-loaded module that reflects these changes, and which the code can invoke
#
def cloneAndMutateModel(inputModelModule, alterations):
    newModelFilename = str(uuid.uuid4())
    newModelFQFilename = DYNAMIC_MODEL_DIR + "/" + newModelFilename + ".py"
    newModelModuleName = DYNAMIC_MODEL_BASE + "." + newModelFilename
    
    alterationD = {}
    for constantLineNo, newConstant in alterations:
        alterationD[constantLineNo-1] = newConstant


    with open(inputModelModule.__file__, "r") as src:
        with open(newModelFQFilename, "w") as dst:
            for lineno, line in enumerate(src):
                if lineno in alterationD:
                    newConstant = alterationD[lineno]
                    lhs, rhs = line.split("=")
                    line = lhs + " = " + str(newConstant) + "\n"
                dst.write(line)

    newModelModule = importlib.import_module(newModelModuleName)

    #
    # MAKE CHANGES TO METADATA
    #
    newAskemMetadata = {}
    newAskemMetadata["ASKEM_ID"] = uuid.uuid4().hex
    newAskemMetadata["ASKEM_CLASS"] = "Thing"
    newAskemMetadata["PROPERTIES"] = [("Name", "Altered Model"),
                                    ("date", datetime.now().strftime("%b %d, %Y %H:%M:%S")),
                                     ("rawfile", newModelModule.__file__)]
    newAskemMetadata["EXTERNAL_URL"] = ""
    newAskemMetadata["DOMAIN_TAGS"] = ""
    newAskemMetadata["RAW_DATA"] = {"text": open(newModelModule.__file__).read()}
    
    newModelModule.__ASKEMMETADATA__ = newAskemMetadata
    return newModelModule


In [ ]:
def visualizeChimeTypeModelOutputs(moduleToInvoke):
    modelMetadata = moduleToInvoke.__ASKEMMETADATA__
    askemId = modelMetadata["ASKEM_ID"]
    askemProperties = modelMetadata["PROPERTIES"]
    pDict = {}
    for k, v in askemProperties:
        pDict[k] = v
    
    d_a, s_a, i_a, r_a, e_a = moduleToInvoke.main()
    modelResults = {"susceptible": s_a,
             "infected": i_a,
             "recovered": r_a,
             "day": d_a,
             "ever_infected": e_a}
    df = pd.DataFrame(modelResults)
    df['day'] = df.index

    fig = px.line(df, 
                  x="day", 
                  y=["susceptible", "infected", "recovered"], 
                  color_discrete_sequence=["black", "red", "green"],
                  title="Model '" + pDict["Name"] + "' (ASKEM_ID:" + askemId + ", date:" + pDict["date"] + ")")
    fig.update_layout(yaxis_title="Population")
    fig.update_layout(xaxis_title="Day")
    return fig


In [ ]:
def loadModelSourceFromXDD(xddid):
    r = requests.get(XDD_GET_URL + xddid)
    if r.status_code == 200:
        robj = r.json()
        if "success" in robj:
            successObj = robj["success"]
            xddObj = successObj["data"][0]

            dataObj = xddObj["data"]
            metadataObj = xddObj["metadata"]
            return dataObj
        else:
            raise Exception("Field 'success' not found in XDD fetched object" + robj)
    else:
        raise Exception("XDD GET failed")


In [ ]:
def computeGrometRepr(lmodule):
    cast = gromet.run_python_to_cast(lmodule.__file__)
    gromet_object = gromet.run_cast_to_gromet_pipeline(cast)
    return gromet_object


In [ ]:
def storeGrometInXDD(grometObj, srcModule):
    gromet_collection_dict = grometObj.to_dict()
    grometJsonRepr = gromet.dictionary_to_gromet_json(gromet.del_nulls(gromet_collection_dict))
    
    xddObj = {
        "metadata": "Blank Metadata Maybe Obsolete",
        "data": {
            "ASKEM_ID": uuid.uuid4().hex,
            "ASKEM_CLASS": "Thing",
            "PROPERTIES": [("Name", "Gromet representation of " + srcModule.__ASKEMMETADATA__["ASKEM_ID"]),
                         ("date", datetime.now().strftime("%b %d, %Y %H:%M:%S"))],
              "DOMAIN_TAGS": [],
              "RAW_DATA": grometJsonRepr,
              "EXTERNAL_URL": ""   
        }
    }
    r = requests.post(XDD_CREATE_URL,
                  headers={"x-api-key": MIT_XDD_KEY,
                          "Content-Type": "application/json"},
                      data=json.dumps(xddObj))  
    if r.status_code == 200:
        return r.json()["success"]["data"]["success"]["registered_ids"][0]
    else:
        raise Exception("Cannot PUT Gromet Object")



In [ ]:
def storeAlteredModelInXDD(curModelModule):
    askemObj = curModelModule.__ASKEMMETADATA__
    xddObj = {
        "metadata": "Blank Metadata Maybe Obsolete",
        "data": askemObj
    }
    r = requests.post(XDD_CREATE_URL,
                    headers={"x-api-key": MIT_XDD_KEY,
                          "Content-Type": "application/json"},
                      data=json.dumps(xddObj))  
    if r.status_code == 200:
        return r.json()["success"]["data"]["success"]["registered_ids"][0]
    else:
        raise Exception("Cannot PUT altered model")


### SKEMA Text Reading Functions

In [ ]:
def get_sentence(mention:Mapping[str, Any], documents:Mapping[str, Any]):
    """ Returns the text containing the mention """
    doc_id = mention['document']
    sentence_ix = mention['sentence']
    sentence = documents[doc_id]['sentences'][sentence_ix]
    return ' '.join(sentence['words'])


def build_groundings(mention:Mapping[str, Any]) -> Optional[str]:
    """ Extracts the grounding ID of the grounding associated to 
    the current mention """
    if mention['type'] == "TextBoundMention":
        groundings = list()
        for a in mention['attachments']:
            # If score is a field in the attachment, then this is a grounding
            if type(a) == list:
                for g in a[0]:
                    score = "{:.2f}".format(g['score'])
                    groundings.append(f"({g['name']}, {g['id']}, {score})")
        if len(groundings) > 0:
            return ', '.join(groundings)


In [ ]:
def annotate_json(file_path:str, endpoint:str = "http://localhost:9000/cosmos_json_to_mentions") -> List[Mapping[str, Any]]:
    """ Annotates an existing json file on the server with the text reading pipeline """

    data = requests.post(endpoint, json={'pathToCosmosJson': file_path}).json()

    return data


def build_data_frame(mentions:List[Mapping[str, Any]], documents:Mapping[str, Any]) -> DataFrame:
    """ Builds a data frame from the output of text reading """

    return DataFrame(
        {
            'text': mention['text'],
            'sentence': get_sentence(mention, documents),
            'start_token': mention['tokenInterval']['start'],
            'end_token': mention['tokenInterval']['end'],
            'mention_type': mention['type'],
            'label': mention['labels'][0],
            'grounding_id': build_groundings(mention),
            # 'sentence_ix': mention['sentence'],
        }
        for mention in mentions
    )

def text_reading_mentions(file_path:str) -> DataFrame:
    """ Puts together annotation and creating the data frame"""

    # return build_data_frame(**annotate_json(file_path))

    return pd.read_csv("tr_cached_results.csv", index_col=0)


In [ ]:
def get_paper_by_doi(doi:str) -> Optional[str]:
    """ Mocks getting the Cosmos output by the paper's DOI """
    papers = {
        "10.1016/j.chaos.2021.110689": "/home/enoriega/documents_5Febcovid19--COSMOS-data.json"
    }
    return papers.get(doi)


# <span style='background :orange' > >>>>> TA-1 Demo Start Here! <<<<< </span>

## Step 0: [Mike 10 sec] Indicate XDD identifier for input model source

In [ ]:
modelSourceXDDId = "761ff74a-b0e1-48bf-b79a-851e3da6b207"


## Step 1: [Mike 20 sec] Import model source code from file, run it, and visualize results

In [ ]:
askemObj = loadModelSourceFromXDD(modelSourceXDDId)
loadedModelModule = storeModelSourceAndLoad(askemObj["RAW_DATA"]["text"], askemObj)
visualizeChimeTypeModelOutputs(loadedModelModule)


## Step 2: [Powell 1 min] VS code environment as IDE for TERArium

A proof of concept has been created to show an in-application IDE in TERArium. This IDE is capable of running the custom VS code extension that we created to annotate code. In addtion, this extension enables the IDE in TERArium to send an entire source-code file to GROMET, receive the resulting GROMET, and highlight code syntax with relevant metadata so the end-user can confirm the GROMET extraction proceeded correctly.

Demo Video - https://youtu.be/WQZiLJUN8H0

## Step 3: [Clay 30 sec] Extract the Gromet Function Network (SKEMA Code2FN) and insert into XDD

Extract the GroMEt Function Network from the source code.

Then insert the FN into XDD, obtaining the XDD ID

In [ ]:

grometObject = computeGrometRepr(loadedModelModule)
grometXDDId = storeGrometInXDD(grometObject, loadedModelModule)
grometXDDId


In [ ]:
grometObject


## Step 4: [Ben 2 min] Load MIRA Epidemiology Domain Ontology Graph

We first set up access to the MIRA Epidemiology Domain Knowledge Graph (DKG) web service.

In [ ]:
MIRA_DKG_URL = 'http://34.230.33.149:8771'

def get_mira_dkg_term(term, attribs):
    res = requests.get(MIRA_DKG_URL + '/api/search', params={'q': term})
    term = [entity for entity in res.json() if entity['id'].startswith('askemo')][0]
    res = {attrib: term.get(attrib) for attrib in attribs if term.get(attrib) is not None}
    return res

Next, we choose a set of terms and attributes to pull into a local ontology that we can use to connect to the model. In addition, for some of the terms, we define custom ranges in which their values for this model can be adjusted.

In [ ]:
# Terms we want to find in MIRA and specific attributes we want to add to our local ontology
terms = ['population', 'doubling time', 'recovery time', 'infectious time']
attribs = ['description', 'synonyms', 'xrefs', 'suggested_unit', 'suggested_data_type',
           'physical_min', 'physical_max', 'typical_min', 'typical_max']

# The local ontology if filled up from the MIRA DKG
LOCAL_ONTOLOGY = {
    term: get_mira_dkg_term(term, attribs) for term in terms
    }

# We can also set further local / use-case specific constraints as needed
LOCAL_ONTOLOGY['population']['typical_min'] = 1000
LOCAL_ONTOLOGY['population']['typical_max'] = 40_000_000

LOCAL_ONTOLOGY

## Step 5: [Ian 2 min] Domain knowledge graph registration and indexing

xDD now has a framework to support ingestion of domain knowledge graphs stored in RDF format. Entities (nodes) from the MIRA knowledge graph have been stored as ASKEM objects (with class=Term), enabling recall within the xDD system:

In [ ]:
url = f"{XDD_GET_URL}?askem_class=Term&description=incidence rate"
resp = requests.get(url)
data = resp.json()
result = data["success"]["data"][:3]
ppr.pprint(result)

This knowledge graph is then indexed and projected into the literature -- currently via a naive term-based matching. This enables summarization of label volume across the xDD corpus (or subsets therein). The example below looks at the hyper-focused modeling-specific subset (more info in the next step), accumulating the number of mentions within the literature of each node label within the ingested DKG.

In [ ]:
url = f"{XDD_ARTICLES_URL}articles?dataset=askem_covid_demo&known_terms=mira_rdf_terms&term_format=full&n_hits=5"
resp = requests.get(url)
data = resp.json()
from collections import defaultdict
term_stats = defaultdict(int)
for document in data["success"]["data"]:
  for dictionary in document['known_terms']:
    for _, terms in dictionary.items():
      for term in terms:
        term_stats[term['term']] += term['n_hits']
ppr.pprint(sorted(term_stats.items(), key=lambda i: -i[1]))

## Step 6: [Ian 2 min] Literature queries and Relevant Corpus Definitions

xDD provides the ability to define _sets_ of documents to process and investigate as a small, targeted corpus. This enables users to reach into the relevant literature to extract text, tables, figures relevant to a query. Embedding models can be trained to enable phrase and document similarity lookups within this targeted set.

Two xDD sets have been defined for this demonstration: One very small, hyper-targeted set (33 documents), compiled through collaboration with Ben Gyori and the MITRE starter kit. The second casts a wider net (~9,000 documents), requiring occurrence of both "covid-19" (or a synonym) and one of a handful of modeling-related terms ["infection rate", "susceptible population", "recovery rate", ...]

### Similar documents
Given a DOI of a document ("Projections and fractional dynamics of COVID-19 with optimal control strategies" by Nabi et. al), we can quicky recovery semantically similar documents within the covid-19 modeling set:


In [ ]:
url = f"https://xdd.wisc.edu/sets/xdd-covid-19-modeling/doc2vec/similar?doi=10.1016/j.chaos.2021.110689"
resp = requests.get(url)
ppr.pprint([f'{i["bibjson"]["title"]} ({i["bibjson"]["identifier"][0]["id"]})'  for i in resp.json()['data']])

### Parameter exploration

xDD allows the ability to search for text mentions of entities and parameters and, via COSMOS extractions, mentions within tables, figures, captions, and linked mentions within body text.

If a modeler is interested in finding more information about γ (recovery rate). They could look up usage of this parameter within related literature. For example, by explicitly looking it up within the SEIRS publication ("The SEIRS model for infectious disease dynamics"):


In [ ]:
url = "https://xdd.wisc.edu/api/articles?title=" + urllib.parse.quote("The SEIRS model for infectious disease dynamics")
# https://xdd.wisc.edu/api/articles?title=The SEIRS model for infectious disease dynamics
resp = requests.get(url)
docid = resp.json()['success']['data'][0]["_gddid"]
url = "https://xdd.wisc.edu/api/snippets?term=" + urllib.parse.quote("γ") + f"&docid={docid}&clean"
# https://xdd.wisc.edu/api/snippets?term=γ&docid=616aca9567467f7269c432ba&clean
resp = requests.get(url)
ppr.pprint(resp.json())

Indicating that a recovery time of 1/γ = 14 days may be a reasonable starting point. However, the user can cast a wider net, and seek tables of relevant values from the very-focused modeling literature:

In [ ]:
url = f"{XDD_GET_URL}?askem_class=Table&contentText=" + urllib.parse.quote("γ")
# https://xdddev.chtc.io/askem/object?askem_class=Table&contentText=γ

resp = requests.get(url)
data = resp.json()
result = data["success"]["data"][2] # Grab the element we want
# alternatively, direct URL: https://xdddev.chtc.io/askem/object/0842faf6-c659-4604-b553-55d8bfbe928d"
display.Image(b64decode(result['properties']['image']))

Part of this response is a dataframe extraction of the discovered table, which may be useful for extraction (or ingestion into the workbench via the dataset cleanup/annotation process under development by Jataware:

In [ ]:
df = pd.read_json(json.dumps(result['properties']['contentJSON']))
df

## Step 7: [Powell 2 min] Table cleanup

A table reading proof of concept has been integrated into the Dojo data annotation stack. This demo shows the capability to place an image of a table and an editable copy of the extracted table into the dojo data registration flow. The end user can make changes to the table and then export it back to xDD.

Demo Video - https://youtu.be/WSn8vlcZbEg

## Step 8: [Enrique 2 min] Perform machine reading of target paper (SKEMA Text Reading Pipeline), Grounding to concepts in the MIRA Epidemiology DKG

### Test Paper Bibliography

__Projections and fractional dynamics of COVID-19 with optimal control strategies__
_Chaos, Solitons & Fractals, Volume 145, April 2021, 110689_

Khondoker Nazmoon Nabi, Pushpendra Kumar, Vedat Suat Erturk

Received 30 November 2020, Revised 9 January 2021, Accepted 12 January 2021, Available online 28 January 2021, Version of Record 2 March 2021.

In [ ]:
pd.set_option('display.max_colwidth', None)

paper_path = get_paper_by_doi("10.1016/j.chaos.2021.110689")

text_reading_mentions(paper_path)

### Sample of relevant, grounded extractions

In [ ]:
pd.read_csv("sample_for_demo.csv", index_col=0)

## Step 9: [Clay 2 min] Analyze Gromet FN MOdel Role Analysis and Exraction (SKEMA MORAE Module)

SKEMA Model Role Analysis and Extraction Module analyzes FN to identify modeling roles. Adapts the [Data Science Ontology framework](https://www.datascienceontology.org/).


Left: Dynamic data flow representation of CHIME SIR model, with code roles highlighted

Right: Map to **Parameter / Initial-Condition / Model / Simulator / Result** pattern

![code_role](images/dso_goal_example.png)


Expansion of "flattened" `sim_sir` function simulator with core `sir` dynamics update functdion; highlighting of core dynamics update

![core_dynamics](images/flat_CHIME_sir_label.png)


### Planed: Automatic extraction of core dynamics BiLayer

This is currently manually constructed. The following is the BiLayer for the CHIME SIR model core dynamics. This is input to TA2.

```
{"Wa":[{"influx":1,"infusion":2},
			 {"influx":2,"infusion":3}],
 "Win":[{"arg":1,"call":1},
 				{"arg":2,"call":1},
 				{"arg":2,"call":2}],
 "Box":[{"parameter":"beta"},
 				{"parameter":"gamma"}],
 "Qin":[{"variable":"S"},
 				{"variable":"I"},
 				{"variable":"R"}],
 "Qout":[{"tanvar":"S'"},
 				 {"tanvar":"I'"},
 				 {"tanvar":"R'"}],
 "Wn":[{"efflux":1,"effusion":1},
 			 {"efflux":2,"effusion":2}]}
```

<div>
<img src="images/CHIME_SIR_dynamics_BiLayer.png" width="200"/>
</div>


## Step 10: [Clay 2 min] Analyze alignment of documentation equations to Gromet FN (SKEMA ISA Module)

SKEMA Eqn2MathML module extracts MathML from documentation images of equations; represent MathML as graph structure.

Perform search to align structure equation and FN structure using Seeded Graph Matching. Seeding comes from term identity based on grounding; reduces the search space.

![S_update_eqn](images/S_update.png)

Graph Matching (no seeds) prior to alignment:
![sgm_adj_marices_pre](images/adjacent_matrices_pre-align.png)

Post alignment; in this case, perfect isomorphism:
![sgm_adj_marices_post](images/adjacent_matrices_aligned.png)

## Step 11: [Dan 2 min] Analyze Gromet FN to identify beta parameter ranges that satisfy constraint (SKEMA FUNMAN)

The Funman module synthesizes the Parameter Space for a model, which encodes feasible parameter settings (denoted "true"), and infeasible parameter settings (denoted "false").  The code in this cell generates the Parameter Space and plots it.

Funman synthesizes parameters with a box search algorithm that encodes a
series of related Satisfiability Modulo Theories (SMT) encodings. The box search involves sub-dividing the space of possible parameter-value combinations to identify boxes that either contain only feasible values or infeasible values for the parameters. If a box contains a feasible point, the solver attempts to prove the absence of infeasible points. If no infeasible points are found, then the box is labeled feasible.  Similarly, if a box contains an infeasible point and no feasible points, it is labeled infeasible. If a box contains both a feasible and infeasible point, then the solver subdivides the box.  The labeling and subdividing process continues until no boxes above a given threshold size remain.

Funman evaluates the extracted model to identify values for beta that:
 - Represent the intervention efficacy over two successive interventions
 - Occur over two time periods 0-20 days (beta_1), and 20-30 days (beta_2)
 - Prevent the number of infected I from exceeding 10% of the population
 - Use a linearized definition for the S and R state update equations
 - Fall within a specified range [0.0, 0.5]

The Parameter Synthesis problem accepts two constants: 
 - population_size: the number of individuals (S+I+R)
 - infectious_days: the number of infected days before recovery

The scenario configuration also supports use of:
 - epochs: the day index ranges corresponding to each beta
 - read_cache_parameter_space: use a precomputed Parameter Space encoded as JSON
 - write_cache_parameter_space: write a precomputed Parameter Space encoded as JSON
 - real_time_plotting: whether to render the solver's progress

In [ ]:
gromet_file1 = "bar"
result1 : ParameterSynthesisScenarioResult = Funman().solve(
    ParameterSynthesisScenario(
        [Parameter("beta_0", lb=0.0, ub=0.5), Parameter("beta_1", lb=0.0, ub=0.5)], 
        gromet_file1,
        config = {
            "epochs": [(0, 20), (20, 30)],
            "population_size": 1002,
            "infectious_days": 14.0, 
            "read_cache_parameter_space" : "result1.json",
            # "write_cache_parameter_space": "result1.json",
            "real_time_plotting": False
        }
    ))

result1.parameter_space.plot()

## Step 12: [Mike 2 min] Analyze Program to identify and annotate model parameters (MIT System)

Analyze Program (eventually: GroMEt Function Network) representation to find connections between FN code and Pandemic Ontology's parameters ("knobs").

In [ ]:
#
# Build Code/Ontology connections based on OpenAI GPT-3
#
# Hard coded version for connection

# targets = ['population', 'doubling time', 'recovery time', 'infectious time']


code = "CHIME_SIR_default_model.py"
terms = list(LOCAL_ONTOLOGY.keys())
targets = ['population', 'infectious time']
discoveredParameterConnections = []
try:
    discoveredParameterConnections = gpt.match_targets(targets, code, terms)
except OpenAIError as err:
    print("OpenAI connection error:", err)
    print("Using hard-coded connections")
    discoveredParameterConnections = [("infectious time", {"name": "grometSubObject"}, 14.0, 67),("population", {"name": "grometSubObject"}, 1000, 80)]

discoveredParameterConnections

## Step 13: [Mike 2 min] Show the user the knobs we found in the code. User can adjust them and recompute

In [ ]:
currentKnobs = {}
sliders = {}
rows = []
rows.append(widgets.HTML("<b>Recovered parameters</b>"))
for dpc in discoveredParameterConnections:
    parameterName, grometObj, curValue, lineOfCode = dpc
    ontologyRecord = LOCAL_ONTOLOGY[parameterName]  # ["parameters"][parameterName]

    currentKnobs[parameterName] = curValue
    curSlider = widgets.IntSlider(min=ontologyRecord["typical_min"], 
                                     max=ontologyRecord["typical_max"], 
                                     value=curValue)

    sliders[parameterName] = curSlider
    
    row = widgets.HBox([widgets.HTML(value="<b>" + parameterName + "</b>"), curSlider])
    tinyCell = widgets.VBox([row, widgets.Label(value=ontologyRecord["description"] + " (observed value: " + str(curValue) + ")")])
    rows.append(tinyCell)

button = widgets.Button(description="Alter Model")
output = widgets.Output()

newModelModules = []

def on_button_clicked(b):
    # rebuild the model and get the result
    edits = []
    for dpc in discoveredParameterConnections:
        parameterName, grometObj, curValue, lineOfCode = dpc
        sliderValue = sliders[parameterName].value
        edits.append((lineOfCode, sliderValue))

    newModelModule = cloneAndMutateModel(loadedModelModule, edits)
    with output:
        output.clear_output()
        newModelModules.clear()
        newModelModules.append(newModelModule)
        display.display(visualizeChimeTypeModelOutputs(newModelModule))
            
button.on_click(on_button_clicked)

rows.append(button)
rows.append(output)
for s in rows:
    display.display(s)

## Step 14: [Dan 2 min] Update FUNMAN beta analysis after reparameterization (SKEMA FUNMAN)

This second use of Funman evaluates the model after changing the population size (10x) and infectious days (0.5x) constants.

In [ ]:
gromet_file2 = "baz"
result2 : ParameterSynthesisScenarioResult = Funman().solve(
    ParameterSynthesisScenario(
        [Parameter("beta_0", lb=0.0, ub=0.5), Parameter("beta_1", lb=0.0, ub=0.5)],
        gromet_file2,
        config = {
            "epochs": [(0, 20), (20, 30)],
            "population_size": 10020,
            "infectious_days": 7.0, 
            "read_cache_parameter_space" : "result2.json",
            # "write_cache_parameter_space": "result2.json",
            "real_time_plotting": False
        }
    ))

result2.parameter_space.plot()

Funman produces two parameter spaces for beta_0 and beta_1.  The parameter spaces overlap (i.e., have a non-empty intersection) and indicate values for the betas that are robust to changes in the model constants.  Similarly, the distinct regions (i.e., symmetric difference) denote parameter values where the models behave differently.

The plot below illustrates:
- The parameter space for the original model constants.
- The parameter space for the modified model constants.

The interpretation of the intersection (dark green, dark red) is that the feasible values for beta denote interventions that scale with population size and the number of infectious days. These values warrant further investigation by the analyst. The interpretation of the symmetric difference is that the feasible values for beta denote interventions that will not scale with population size and the number of infectious days.  These values may be discarded by the analyst.

In [ ]:
result1.parameter_space.plot()
result2.parameter_space.plot()

## Step 15: [Ian 30 sec] When the user obtains an altered model she likes, she can choose to store it in XDD, along with metadata about how it was created

xDD has been extended to include rudimentary registration, recall, and searching of JSON objects. This registration is backed by a shared metadata schema, which serves as a sort of a data contract -- as long as the registered objects match the agreed upon schema, they will be indexed and queryable.

A primary benefit of xDD being the datastore is the potential benefit for synthesis and data integration within one system. The vision is to be able to bring back document, parameter, table, and structured datasets relevant to a query in one shot, rather than having to access multiple endpoints across the ecosystem.

A secondary benefit is having a central place to assign stable object identities that can be used throughout the ASKEM ecosystem, with one place for recalling the object.

The current status is that it is a viable proof of concept, with initial indexing on a handful of critical terms.

Short term improvement targets are updates to the recall API (more search options, pagination) and schema adherence checking.

Longer term development plans include a versioning system to track and return updates and annotations over time, while returning the latest version of an object by default.

In [ ]:
if len(newModelModules) > 0:
    print("XDD ID: ", storeAlteredModelInXDD(newModelModules[-1]))